<a href="https://colab.research.google.com/github/JK-the-Ko/Thermo-Fluid-Dynamics-Experiment/blob/main/2022-2/%EC%97%B4%EC%9C%A0%EC%B2%B4%EA%B3%B5%ED%95%99%EC%8B%A4%ED%97%98_Week_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Data Analysis

## Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Harmonic Function

In [ ]:
x = np.sin(np.arange(0, 1e3, 1e-1))

## Data Processing for Neural Network

### Get Number of Input

In [ ]:
numInput = 50
numOutput = 25

### Add Data

In [ ]:
inputData, targetData = [], []

for i in range(len(x)-numInput-numOutput) :
  inputData.append(x[i:i+numInput])
  targetData.append(x[i+numInput:i+numInput+numOutput])

In [ ]:
inputData, targetData = np.array(inputData), np.array(targetData)

### Show Dataset

In [ ]:
inputDF = pd.DataFrame(data=inputData, columns=[f"Input_{i}" for i in range(numInput)])
targetDF = pd.DataFrame(data=targetData, columns=[f"Target{i}" for i in range(numOutput)])

In [ ]:
inputDF.head(5)

In [ ]:
targetDF.head(5)

### Split Dataset

In [ ]:
trainRatio = 0.8
trainSize = int(inputData.shape[0]*trainRatio)
validSize = (inputData.shape[0]-trainSize)//2

In [ ]:
trainSize

In [ ]:
xTrain, yTrain = inputData[:trainSize], targetData[:trainSize]
xValid, yValid = inputData[trainSize:trainSize+validSize], targetData[trainSize:trainSize+validSize]
xTest, yTest = inputData[trainSize+validSize:], targetData[trainSize+validSize:]

## Recurrent Neural Network (RNN)

### Import Keras Library

In [ ]:
from tensorflow import keras

### Fix Seed

In [ ]:
import tensorflow as tf
from keras import backend as K
import random

def fixSeed(numSeed = 42) :
  np.random.seed(numSeed)
  random.seed(numSeed)
  tf.random.set_seed(numSeed)

  sessionConf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=sessionConf)
  K.set_session(sess)

### Create RNN Network

In [ ]:
K.clear_session()
fixSeed()

model = keras.Sequential()

model.add(keras.layers.SimpleRNN(50, input_shape=(numInput, 1)))
model.add(keras.layers.Dense(numOutput))

### Summarize Model

In [ ]:
model.summary()

### Visualize Model

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file="model.png")

### Compile Model

In [ ]:
lr, batchSize, epoch = 1e-3, 1024, 100

In [ ]:
model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(learning_rate=lr))

### Train Model

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epoch, validation_data=(xValid, yValid))

### Visualize Training Procedure

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(np.arange(epoch), history.history["loss"], label="Training MAE Loss")
plt.plot(np.arange(epoch), history.history["val_loss"], label="Validation MAE Loss")
plt.xlabel("# Epoch")
plt.ylabel("MAE Loss")
plt.title("Loss")
plt.legend(loc="best")
plt.show()

### Inference Result

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

### Visualize Result

In [ ]:
plt.subplots(figsize = (20,5))
plt.scatter(np.arange(numInput), xTest[-1].reshape(-1), label="Input", marker=".")
plt.scatter(np.arange(numInput, numInput+numOutput), yTest[-1].reshape(-1), label="Ground Truth", marker="x")
plt.scatter(np.arange(numInput, numInput+numOutput), yPred[-1].reshape(-1), label="Prediction", marker="^")
plt.xlabel("Time Step (0.1s)")
plt.ylabel("Output")
plt.title("Result Comparison")
plt.legend(loc="best")
plt.show()

### Model Evaluation

In [ ]:
def computeMAE(pred, target) :
  return np.abs(pred-target).mean()

In [ ]:
computeMAE(yPred, yTest)

## Create MCK System Class

In [ ]:
class MCKSystem :
  def __init__(self, mass, springStiffness, dampingCoefficient) :
    # Initialize Variable
    self.mass = mass
    self.springStiffness = springStiffness
    self.dampingCoefficient = dampingCoefficient
    
    # Compute Varaible
    self.naturalFrequency = self.computeNaturalFrequency()
    self.dampingRatio = self.computeDampingRatio()
    self.dampedNaturalFrequency = self.computeDampedNaturalFrequency()

  def computeNaturalFrequency(self) :
    return np.sqrt(self.springStiffness/self.mass)  

  def computeDampingRatio(self) :
    return self.dampingCoefficient/(2*self.mass*self.naturalFrequency)

  def computeDampedNaturalFrequency(self) :
    return self.naturalFrequency*np.sqrt(1-np.power(self.dampingRatio, 2))

  def computeDisplacement(self, x0, v0, t) :
    return np.exp(-self.dampingRatio*self.naturalFrequency*t)*(x0*np.cos(self.dampedNaturalFrequency*t) + ((v0+self.dampingRatio*self.naturalFrequency*x0)/self.dampedNaturalFrequency)*np.sin(self.dampedNaturalFrequency*t))

## Create Object Instance

In [ ]:
m, c, k = 1e3, 10, 10
mySystem = MCKSystem(m, c, k)

## Get Initial Condition and Time

In [ ]:
x0, v0 = 0, 1
t = np.arange(0, 1e3, 1e-1)

## Compute Displacement

In [ ]:
x = mySystem.computeDisplacement(x0, v0, t)

## Plot Graph

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(t, x)
plt.xlabel("Time (0.1s)")
plt.ylabel("Displacement (m)")
plt.title("MCK System")
plt.show()

## Data Processing for Neural Network

### Get Number of Input

In [ ]:
numInput = 500
numOutput = 500

### Add Data

In [ ]:
inputData, targetData = [], []

for i in range(len(x)-numInput-numOutput) :
  inputData.append(x[i:i+numInput])
  targetData.append(x[i+numInput:i+numInput+numOutput])

In [ ]:
inputData, targetData = np.array(inputData), np.array(targetData)

### Show Dataset

In [ ]:
inputDF = pd.DataFrame(data=inputData, columns=[f"Input_{i}" for i in range(numInput)])
targetDF = pd.DataFrame(data=targetData, columns=[f"Target{i}" for i in range(numOutput)])

In [ ]:
inputDF.head(5)

In [ ]:
targetDF.head(5)

### Split Dataset

In [ ]:
trainRatio = 0.8
trainSize = int(inputData.shape[0]*trainRatio)
validSize = (inputData.shape[0]-trainSize)//2

In [ ]:
trainSize

In [ ]:
xTrain, yTrain = inputData[:trainSize], targetData[:trainSize]
xValid, yValid = inputData[trainSize:trainSize+validSize], targetData[trainSize:trainSize+validSize]
xTest, yTest = inputData[trainSize+validSize:], targetData[trainSize+validSize:]

## Recurrent Neural Network (RNN)

### Create RNN Network

In [ ]:
K.clear_session()
fixSeed()

model = keras.Sequential()

model.add(keras.layers.SimpleRNN(50, input_shape=(numInput, 1)))
model.add(keras.layers.Dense(numOutput))

### Summarize Model

In [ ]:
model.summary()

### Visualize Model

In [ ]:
plot_model(model, to_file="model.png")

### Compile Model

In [ ]:
lr, batchSize, epoch = 1e-3, 1024, 50

In [ ]:
model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(learning_rate=lr))

### Train Model

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epoch, validation_data=(xValid, yValid))

### Visualize Training Procedure

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(np.arange(epoch), history.history["loss"], label="Training MAE Loss")
plt.plot(np.arange(epoch), history.history["val_loss"], label="Validation MAE Loss")
plt.xlabel("# Epoch")
plt.ylabel("MAE Loss")
plt.title("Loss")
plt.legend(loc="best")
plt.show()

### Inference Result

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

### Visualize Result

In [ ]:
plt.subplots(figsize = (20,5))
plt.scatter(np.arange(numInput), xTest[-1].reshape(-1), label="Input", marker=".")
plt.scatter(np.arange(numInput, numInput+numOutput), yTest[-1].reshape(-1), label="Ground Truth", marker="x")
plt.scatter(np.arange(numInput, numInput+numOutput), yPred[-1].reshape(-1), label="Prediction", marker="^")
plt.xlabel("Time Step (0.1s)")
plt.ylabel("Displacement (m)")
plt.title("Result Comparison")
plt.legend(loc="best")
plt.show()

### Model Evaluation

In [ ]:
computeMAE(yPred, yTest)

## Long Short-Term Memory (LSTM)

### Create LSTM Network

In [ ]:
K.clear_session()
fixSeed()

model = keras.Sequential()

model.add(keras.layers.LSTM(50, input_shape=(numInput, 1)))
model.add(keras.layers.Dense(numOutput))

### Summarize Model

In [ ]:
model.summary()

### Visualize Model

In [ ]:
plot_model(model, to_file="model.png")

### Compile Model

In [ ]:
lr, batchSize, epoch = 1e-3, 1024, 50

In [ ]:
model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(learning_rate=lr))

### Train Model

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epoch, validation_data=(xValid, yValid))

### Visualize Training Procedure

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(np.arange(epoch), history.history["loss"], label="Training MAE Loss")
plt.plot(np.arange(epoch), history.history["val_loss"], label="Validation MAE Loss")
plt.xlabel("# Epoch")
plt.ylabel("MAE Loss")
plt.title("Loss")
plt.legend(loc="best")
plt.show()

### Inference Result

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

### Visualize Result

In [ ]:
plt.subplots(figsize = (20,5))
plt.scatter(np.arange(numInput), xTest[-1].reshape(-1), label="Input", marker=".")
plt.scatter(np.arange(numInput, numInput+numOutput), yTest[-1].reshape(-1), label="Ground Truth", marker="x")
plt.scatter(np.arange(numInput, numInput+numOutput), yPred[-1].reshape(-1), label="Prediction", marker="^")
plt.xlabel("Time Step (0.1s)")
plt.ylabel("Displacement (m)")
plt.title("Result Comparison")
plt.legend(loc="best")
plt.show()

### Model Evaluation

In [ ]:
computeMAE(yPred, yTest)

## Gated Recurrent Unit (GRU)

### Create GRU Network

In [ ]:
K.clear_session()
fixSeed()

model = keras.Sequential()

model.add(keras.layers.GRU(50, input_shape=(numInput, 1)))
model.add(keras.layers.Dense(numOutput))

### Summarize Model

In [ ]:
model.summary()

### Visualize Model

In [ ]:
plot_model(model, to_file="model.png")

### Compile Model

In [ ]:
lr, batchSize, epoch = 1e-3, 1024, 50

In [ ]:
model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(learning_rate=lr))

### Train Model

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epoch, validation_data=(xValid, yValid))

### Visualize Training Procedure

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(np.arange(epoch), history.history["loss"], label="Training MAE Loss")
plt.plot(np.arange(epoch), history.history["val_loss"], label="Validation MAE Loss")
plt.xlabel("# Epoch")
plt.ylabel("MAE Loss")
plt.title("Loss")
plt.legend(loc="best")
plt.show()

### Inference Result

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

### Visualize Result

In [ ]:
plt.subplots(figsize = (20,5))
plt.scatter(np.arange(numInput), xTest[-1].reshape(-1), label="Input", marker=".")
plt.scatter(np.arange(numInput, numInput+numOutput), yTest[-1].reshape(-1), label="Ground Truth", marker="x")
plt.scatter(np.arange(numInput, numInput+numOutput), yPred[-1].reshape(-1), label="Prediction", marker="^")
plt.xlabel("Time Step (0.1s)")
plt.ylabel("Displacement (m)")
plt.title("Result Comparison")
plt.legend(loc="best")
plt.show()

### Model Evaluation

In [ ]:
computeMAE(yPred, yTest)

## Deeper Long Short-Term Memory (LSTM)

### Create LSTM Network

In [ ]:
K.clear_session()
fixSeed()

model = keras.Sequential()

model.add(keras.layers.LSTM(100, input_shape=(numInput, 1)))
model.add(keras.layers.Dense(numOutput))

### Summarize Model

In [ ]:
model.summary()

### Visualize Model

In [ ]:
plot_model(model, to_file="model.png")

### Compile Model

In [ ]:
lr, batchSize, epoch = 1e-3, 1024, 50

In [ ]:
model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(learning_rate=lr))

### Train Model

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epoch, validation_data=(xValid, yValid))

### Visualize Training Procedure

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(np.arange(epoch), history.history["loss"], label="Training MAE Loss")
plt.plot(np.arange(epoch), history.history["val_loss"], label="Validation MAE Loss")
plt.xlabel("# Epoch")
plt.ylabel("MAE Loss")
plt.title("Loss")
plt.legend(loc="best")
plt.show()

### Inference Result

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

### Visualize Result

In [ ]:
plt.subplots(figsize = (20,5))
plt.scatter(np.arange(numInput), xTest[-1].reshape(-1), label="Input", marker=".")
plt.scatter(np.arange(numInput, numInput+numOutput), yTest[-1].reshape(-1), label="Ground Truth", marker="x")
plt.scatter(np.arange(numInput, numInput+numOutput), yPred[-1].reshape(-1), label="Prediction", marker="^")
plt.xlabel("Time Step (0.1s)")
plt.ylabel("Displacement (m)")
plt.title("Result Comparison")
plt.legend(loc="best")
plt.show()

### Model Evaluation

In [ ]:
computeMAE(yPred, yTest)

# 실습
## **AirQualityUCI.csv를 기반으로 시계열 분석을 진행하세요.**
### **1) 데이터 셋을 불러온 후 훈련, 검증, 시험 데이터 셋으로 분리하세요.**
### 1.1) 24개의 데이터를 사용하여 향후 6개의 데이터를 예측
### 1.2) 입력 데이터 → ["Date", "Time"]을 제외한 모든 Feature
### 1.3) 목표 데이터 → ["C6H6(GT)"]
### 1.4) 훈련 데이터 셋 비율 → 0.8
### 1.5) 검증 데이터 셋 비율 → 0.1
### 1.6) 시험 데이터 셋 비율 → 0.1
### **2) Unit이 50인 RNN 모델을 만드세요.**
### **3) 다음과 같은 하이퍼파라미터를 사용하여 모델을 훈련하세요.**
### 3.1) Learning Rate → 1e-3
### 3.2) Batch Size → 256
### 3.3) Epochs → 50
### **4) 시험 데이터 셋을 사용하여 모델을 평가하세요**
